### Importación de librerías

In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from scipy.optimize import brentq
from IPython.display import display, Markdown, clear_output

Cargar y organizar CSV

In [22]:
# Cargar y limpiar los datos del CSV
df = pd.read_csv("Flujos_de_caja.csv", sep=";", encoding="latin1")
df.columns = [col.replace('\x96', 'n').replace('A\x96o', 'Año') for col in df.columns]

# Convertir columnas numéricas correctamente
df.iloc[:, 1:] = df.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')

# Verificar y mostrar valores nulos
nulos = df.isnull().sum()

### Funciones Principales

In [28]:
# Funcion para visualizar datos
def visualizar_datos():
    clear_output()
    display(Markdown("### Vista previa de los datos"))
    display(df.head())
    display(Markdown("### Valores nulos por columna:"))
    display(nulos[nulos > 0])
    display(Markdown("### Descripción estadística:"))
    display(df.describe())

# Función auxiliar para calcular la TIR
def calcular_tir(flujos):
    try:
        return brentq(lambda r: np.sum([f / (1 + r) ** i for i, f in enumerate(flujos)]), -0.99, 1.0)
    except:
        return np.nan

# Función para calcular VPN y TIR por proyecto
def calcular_vpn_tir(tasa_descuento=0.1):
    clear_output()
    display(Markdown(f"### Cálculo de VPN y TIR con tasa de descuento del {tasa_descuento*100:.2f}%"))
    resultados = []

    for idx, row in df.iterrows():
        nombre = row["Proyecto"]
        flujos = [-abs(row["Inversion_Inicial"])] + list(row[2:].fillna(0).values)

        # Cálculo del VPN
        vpn = sum([flujo / ((1 + tasa_descuento) ** i) for i, flujo in enumerate(flujos)])

        # Cálculo de la TIR con función personalizada
        tir = calcular_tir(flujos)

        resultados.append({
            "Proyecto": nombre,
            "VPN": vpn,
            "TIR": tir
        })

    res_df = pd.DataFrame(resultados)
    display(res_df)
    return res_df

# Funcion para Graficar Resultados
def graficar_resultados():
    clear_output()
    res_df = calcular_vpn_tir(tasa_descuento_slider.value)

    # Seleccionar los 15 proyectos con mayor VPN
    top_vpn = res_df.sort_values(by="VPN", ascending=False).head(15)

    # Gráfico de VPN
    display(Markdown("### Top 15 Proyectos con Mayor VPN"))
    plt.figure(figsize=(12, 6))
    plt.bar(top_vpn["Proyecto"], top_vpn["VPN"], color='skyblue')
    plt.xticks(rotation=45, ha='right')
    plt.ylabel("VPN")
    plt.title("Valor Presente Neto (VPN)")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # Seleccionar los 15 proyectos con mayor TIR
    top_tir = res_df.sort_values(by="TIR", ascending=False).head(15)

    # Gráfico de TIR
    display(Markdown("### Top 15 Proyectos con Mayor TIR"))
    plt.figure(figsize=(12, 6))
    plt.bar(top_tir["Proyecto"], top_tir["TIR"] * 100, color='orange')
    plt.xticks(rotation=45, ha='right')
    plt.ylabel("TIR (%)")
    plt.title("Tasa Interna de Retorno (TIR)")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

### Menú Interactivo

In [29]:
tasa_descuento_slider = widgets.FloatSlider(value=0.1, min=0.01, max=0.3, step=0.01, description='Descuento:')

menu = widgets.ToggleButtons(
    options=['Visualizar Datos', 'Calcular VPN/TIR', 'Graficar Resultados'],
    description='Selecciona:',
    button_style='info'
)

output = widgets.Output()

def ejecutar_menu(cambio):
    with output:
        if cambio['new'] == 'Visualizar Datos':
            visualizar_datos()
        elif cambio['new'] == 'Calcular VPN/TIR':
            calcular_vpn_tir(tasa_descuento_slider.value)
        elif cambio['new'] == 'Graficar Resultados':
            graficar_resultados()

menu.observe(ejecutar_menu, names='value')

display(Markdown("## Examen Institucional - Análisis Financiero de Proyectos"))
display(tasa_descuento_slider)
display(menu)
display(output)

## Examen Institucional - Análisis Financiero de Proyectos

FloatSlider(value=0.1, description='Descuento:', max=0.3, min=0.01, step=0.01)

ToggleButtons(button_style='info', description='Selecciona:', options=('Visualizar Datos', 'Calcular VPN/TIR',…

Output()